In [70]:
import sys
import json
import socket
from confluent_kafka import Consumer, Producer
from confluent_kafka.cimpl import KafkaException, KafkaError

class WebScrapingProducer:

    def __init__(self):
        conf = {'bootstrap.servers': "localhost:9092,localhost:9092",
                'client.id': socket.gethostname()}
        self.__producer = Producer(conf)

    def criar_mensagem_producer(self, topic, dados_empresa):
        dados_empresa = json.dumps(dados_empresa).encode('utf-8')
        self.__producer.produce(topic=topic,
                                key="key",
                                value=dados_empresa,
                                callback=WebScrapingProducer.__retorno_resultado_mensagem_producer)
        # Wait up to 1 second for events. Callbacks will be invoked during
        # this method call if the message is acknowledged.
        self.__producer.poll(1)

    @staticmethod
    def __retorno_resultado_mensagem_producer(err, dados_empresa):
        if err is not None:
            print("Failed to deliver message: %s: %s" % (str(dados_empresa), str(err)))
        else:
            print("Mensagem criada")

In [71]:
producer = WebScrapingProducer()

In [72]:
retorno = producer.criar_mensagem_producer("teste", {'nome':'Hugo', 'idade':30})

Mensagem criada


In [73]:
import sys
from confluent_kafka import Consumer
from confluent_kafka.cimpl import KafkaException, KafkaError


class WebScrapingConsumer:

    def __init__(self, topics):
        self.__topics = topics
        conf = {'bootstrap.servers': "localhost:9092,localhost:9092",
                'group.id': "foo",
                'auto.offset.reset': 'smallest'}
        self.__consumer = Consumer(conf)

    def mensagem_dados_empresa_consumer(self):
        try:
            self.__consumer.subscribe(self.__topics)
            mensagem = self.__consumer.poll(timeout=1.0)
            if mensagem is None:
                return None
            if mensagem.error():
                if mensagem.error().code() == KafkaError._PARTITION_EOF:
                    # End of partition event
                    sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                        (mensagem.topic(), mensagem.partition(), mensagem.offset()))
                elif mensagem.error():
                    raise KafkaException(mensagem.error())
            else:
                return json.loads(mensagem.value().decode('utf-8'))
        finally:
            self.__consumer.close()

    def shutdown(self):
        running = False

In [74]:
consumer = WebScrapingConsumer(["teste"])

In [75]:
consumer.mensagem_dados_empresa_consumer()

{'nome': 'Hugo', 'idade': 30}